# Connect Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
%cd /content/gdrive/MyDrive/badminton/annotation_pipeline/player

/content/gdrive/MyDrive/badminton/annotation_pipeline/player


# Package Installation
Instructions from https://mmpose.readthedocs.io/en/latest/install.html#installation 

In [13]:
!pip3 install openmim
!mim install mmcv-full

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html


In [15]:
#!git clone https://github.com/open-mmlab/mmpose.git # only clone once to save the repo
%cd mmpose
!pip install -e .

Cloning into 'mmpose'...
remote: Enumerating objects: 22811, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 22811 (delta 82), reused 106 (delta 61), pack-reused 22669
Receiving objects: 100% (22811/22811), 26.49 MiB | 9.17 MiB/s, done.
Resolving deltas: 100% (16235/16235), done.
Checking out files: 100% (1638/1638), done.
/content/gdrive/MyDrive/badminton/annotation_pipeline/player/mmpose
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/MyDrive/badminton/annotation_pipeline/player/mmpose
     |████████████████████████████████| 50 kB 4.0 MB/s 
     |████████████████████████████████| 276 kB 34.2 MB/s 
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58285 sha256=9ba9ca1157ee0d43b050fc032d098a137012da6c99b5e8a52b177f10ce8fe1d1
  Stored in directory: /root/.cache/pip/wheels/59/68/de/5e0c5d77e573e8c150e69e07a25035e6b6a

In [16]:
import mmpose
print(mmpose.__version__)

0.29.0


## For mmdet

In [17]:
!pip3 install mmdet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 21.1 MB/s 


# Top-down pose detection (mmdet + mmpose)
mmdet: faster_rcnn_r50_fpn_coco.py

mmpose: hrnet_w48_coco_384x288.py

## Import packages

In [18]:
import os
import warnings
from argparse import ArgumentParser

from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_result)
from mmpose.datasets import DatasetInfo

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import mmcv

## Detector configuration and parameters

In [20]:
device = 'cuda:0' # or cpu

# mmdet
det_config = 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# mmpose
pose_config = "configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_384x288.py"
pose_checkpoint = "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_384x288-314c8528_20200708.pth"

## Init detectors

In [21]:
det_model = init_detector(det_config, det_checkpoint, device=device.lower())
pose_model = init_pose_model(pose_config, pose_checkpoint, device=device.lower())

load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


Downloading: "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth" to /root/.cache/torch/hub/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_384x288-314c8528_20200708.pth


Downloading: "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_384x288-314c8528_20200708.pth" to /root/.cache/torch/hub/checkpoints/hrnet_w48_coco_384x288-314c8528_20200708.pth


  0%|          | 0.00/243M [00:00<?, ?B/s]

## Function for running detection on input video

In [46]:
def topdown_posedet_vid(input_vidname, output_vidname, out_csv, det_model, pose_model, bbox_thr=0.5, kpt_thr=0.3, save_outvid=False):
  '''
  saves bbox and pose detections in a csv, with columns frame, bbox (2x2), bbox_conf, pose (17x2)
  '''

  dataset = pose_model.cfg.data['test']['type']
  dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
  if dataset_info is None:
      warnings.warn(
          'Please set `dataset_info` in the config.'
          'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
          DeprecationWarning)
  else:
      dataset_info = DatasetInfo(dataset_info)
  
  rows_list = []

  video = mmcv.VideoReader(input_vidname)
  cap = cv2.VideoCapture(input_vidname)
  fnum_tot = int(cap.get(cv2. CAP_PROP_FRAME_COUNT))
  h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
  fps = int(cap.get(cv2.CAP_PROP_FPS))

  if save_outvid:
    video_writer = cv2.VideoWriter(output_vidname, cv2.VideoWriter_fourcc(*'MP4V'), fps, (w, h))

  for fnum, img in enumerate(mmcv.track_iter_progress(video)):
    # resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, img)

    # keep the person class bounding boxes.
    person_results = process_mmdet_results(mmdet_results, 1) # 1 is the cat_id of person class

    # get pose_results from inference, in format of list of dicts, each list is one det, with key 'bbox', 'keypoint'
    # seee https://mmpose.readthedocs.io/en/latest/api.html?highlight=inference_top_down_pose_model#mmpose.apis.inference_top_down_pose_model
    pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        img,
        person_results,
        bbox_thr=bbox_thr,
        format='xyxy',
        dataset=dataset,
        dataset_info=dataset_info,
        return_heatmap=False,
        outputs=None)
      
    for person in pose_results:
      pose = person['keypoints']
      bbox = person['bbox']
      row_dict = {}
      row_dict['Frame'] = fnum
      row_dict['x1'], row_dict['y1'], row_dict['x2'], row_dict['y2'], row_dict['bbox_conf'] = bbox[0]/w, bbox[1]/h, bbox[2]/w, bbox[3]/h, bbox[4]
      for i in range(17):
        row_dict[str(i) + '_x'] = pose[i][0]/w
        row_dict[str(i) + '_y'] = pose[i][1]/h
      
      rows_list.append(row_dict)
    
    if save_outvid:
      img_pose = vis_pose_result(pose_model, img, pose_results, show=False)
      img_pose = cv2.resize(img_pose, (w, h), interpolation=cv2.INTER_LANCZOS4)
      
      video_writer.write(img_pose)

  if save_outvid:
    video_writer.release()
    cap.release()

  df = pd.DataFrame(rows_list)
  df.to_csv(out_csv, index=False)
  return df

## Run detection on list of videos

In [42]:
input_vidname = '/content/gdrive/MyDrive/badminton/datasets/am_singles/china_match2/rally_video/singles0.mp4'
out_dir = '/content/gdrive/MyDrive/badminton/annotation_pipeline/player/pose_bbox_am_singles/china_match2/'
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
basename = input_vidname.split('/')[-1][:-4]
out_csv = os.path.join(out_dir, basename + '_pose_bbox.csv')
output_vidname = ''

df_pred = topdown_posedet_vid(input_vidname, output_vidname, out_csv, det_model, pose_model, bbox_thr=0.5, kpt_thr=0.3, save_outvid=False)
df_pred.head(10)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 282/282, 3.1 task/s, elapsed: 91s, ETA:     0s


,Frame,x1,y1,x2,y2,bbox_conf,0_x,0_y,1_x,1_y,...,12_x,12_y,13_x,13_y,14_x,14_y,15_x,15_y,16_x,16_y
0,0,0.504816,0.445466,0.537905,0.623110,0.995694,0.517782,0.460848,0.519734,0.457378,...,0.514529,0.533710,0.526890,0.571876,0.513228,0.569563,0.525589,0.606572,0.513879,0.601946
1,0,0.179344,0.305294,0.311531,0.720942,0.991674,0.230977,0.333166,0.229455,0.330460,...,0.246199,0.514471,0.278164,0.584828,0.246199,0.609182,0.285774,0.679540,0.192923,0.666009
2,0,0.607094,0.516492,0.631961,0.596344,0.845373,0.620259,0.531724,0.621136,0.526526,...,0.614995,0.563436,0.621136,0.561877,0.612363,0.556678,0.621721,0.588390,0.621136,0.590469
3,1,0.501675,0.442657,0.540808,0.623342,0.996720,0.518264,0.460655,0.519588,0.457126,...,0.514294,0.533588,0.526866,0.571230,0.512309,0.570054,0.526204,0.605344,0.511647,0.601815
4,1,0.161730,0.305934,0.310043,0.718850,0.976701,0.216985,0.333623,0.216985,0.330935,...,0.236643,0.516424,0.274446,0.583631,0.229082,0.615890,0.280495,0.675031,0.182206,0.677720
5,1,0.607091,0.516468,0.631968,0.596376,0.842406,0.620261,0.531711,0.621139,0.526509,...,0.614408,0.566046,0.621139,0.561364,0.612652,0.556682,0.621724,0.588416,0.621139,0.590497
6,2,0.501110,0.444547,0.541346,0.616883,0.996663,0.515864,0.461713,0.517757,0.458347,...,0.513339,0.534642,0.527224,0.571667,0.511446,0.571667,0.525961,0.605327,0.507659,0.601961
7,2,0.153976,0.313438,0.277952,0.721721,0.967156,0.200265,0.343474,0.200265,0.340816,...,0.218207,0.524225,0.261567,0.572071,0.216712,0.619916,0.248111,0.665104,0.171856,0.689027
8,2,0.607096,0.516481,0.631957,0.596422,0.843886,0.620258,0.531730,0.621136,0.526526,...,0.614989,0.563477,0.621136,0.561396,0.612354,0.556712,0.621722,0.588459,0.621136,0.590541
9,3,0.497847,0.445729,0.540939,0.620630,0.996929,0.515870,0.464289,0.517151,0.460873,...,0.512668,0.536026,0.526759,0.573602,0.510746,0.572464,0.525478,0.605485,0.506263,0.600930


### am-singles

In [47]:
dataset_rootdir = '/content/gdrive/MyDrive/badminton/datasets/am_singles/' 
outcsv_rootdir = '/content/gdrive/MyDrive/badminton/annotation_pipeline/player/pose_bbox_am_singles/'
match_dirs = [matchdir for matchdir in sorted(os.listdir(dataset_rootdir))]

for matchdir in match_dirs:
  out_dir = os.path.join(outcsv_rootdir, matchdir)
  if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
  viddir = os.path.join(dataset_rootdir, matchdir,'rally_video')
  for vid in os.listdir(viddir):
    if vid.endswith(".mp4"):
      input_vidname = os.path.join(viddir, vid)
      print(input_vidname)
      out_csv = os.path.join(out_dir, vid[:-4] + '_pose_bbox.csv')
      output_vidname=''

      df_pred = topdown_posedet_vid(input_vidname, output_vidname, out_csv, det_model, pose_model, bbox_thr=0.6, kpt_thr=0.3, save_outvid=False)

/content/gdrive/MyDrive/badminton/datasets/am_singles/match24/rally_video/1_00_01.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 200/200, 3.8 task/s, elapsed: 52s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/am_singles/match24/rally_video/1_01_01.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 189/189, 3.9 task/s, elapsed: 49s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/am_singles/match24/rally_video/1_01_02.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 443/443, 3.9 task/s, elapsed: 113s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/am_singles/match24/rally_video/1_01_03.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 313/313, 3.9 task/s, elapsed: 81s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/am_singles/match24/rally_video/1_01_04.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 243/243, 3.9 task/s, elapsed: 62s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/am_singles/match24/rally_video/1_02_04.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 714/714, 3.9 task/s, elapsed: 183s, ETA:     0s
/con

### pro

In [ ]:
dataset_rootdir = '/content/gdrive/MyDrive/badminton/datasets/pro/' 
outcsv_rootdir = '/content/gdrive/MyDrive/badminton/annotation_pipeline/player/pose_bbox_pro/'
#match_dirs = [matchdir for matchdir in sorted(os.listdir(dataset_rootdir))]
match_dirs_done = ['match1', 'match10', 'match11', 'match12', 'match13', 'match14', 'match15', 'match16', 'match17', 'match18', 'match19', 'match2', 'match20', 'match21']
match_dirs = [matchdir for matchdir in sorted(os.listdir(dataset_rootdir)) if matchdir not in match_dirs_done]

for matchdir in match_dirs:
  out_dir = os.path.join(outcsv_rootdir, matchdir)
  if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
  viddir = os.path.join(dataset_rootdir, matchdir,'rally_video')
  for vid in os.listdir(viddir):
    if vid.endswith(".mp4"):
      input_vidname = os.path.join(viddir, vid)
      print(input_vidname)
      out_csv = os.path.join(out_dir, vid[:-4] + '_pose_bbox.csv')
      output_vidname=''

      df_pred = topdown_posedet_vid(input_vidname, output_vidname, out_csv, det_model, pose_model, bbox_thr=0.6, kpt_thr=0.3, save_outvid=False)

/content/gdrive/MyDrive/badminton/datasets/pro/match1/rally_video/1_03_05.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 359/359, 1.9 task/s, elapsed: 190s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/pro/match1/rally_video/1_01_00.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 651/651, 1.9 task/s, elapsed: 343s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/pro/match1/rally_video/1_02_00.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 500/500, 1.9 task/s, elapsed: 262s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/pro/match1/rally_video/1_02_01.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 196/196, 1.9 task/s, elapsed: 103s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/pro/match1/rally_video/1_02_02.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 259/259, 1.8 task/s, elapsed: 143s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/pro/match1/rally_video/1_02_03.mp4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 80/80, 1.8 task/s, elapsed: 44s, ETA:     0s
/content/gdrive/MyDrive/badminton/datasets/pro/match

### am_doubles

In [ ]:
dataset_rootdir = '/content/gdrive/MyDrive/badminton/datasets/am_doubles/' 
outcsv_rootdir = '/content/gdrive/MyDrive/badminton/annotation_pipeline/player/pose_bbox_am_doubles/'
match_dirs = [matchdir for matchdir in sorted(os.listdir(dataset_rootdir))]

for matchdir in match_dirs:
  out_dir = os.path.join(outcsv_rootdir, matchdir)
  if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
  viddir = os.path.join(dataset_rootdir, matchdir,'rally_video')
  for vid in os.listdir(viddir):
    if vid.endswith(".mp4"):
      input_vidname = os.path.join(viddir, vid)
      print(input_vidname)
      out_csv = os.path.join(out_dir, vid[:-4] + '_pose_bbox.csv')
      output_vidname=''

      df_pred = topdown_posedet_vid(input_vidname, output_vidname, out_csv, det_model, pose_model, bbox_thr=0.6, kpt_thr=0.3, save_outvid=False)